In [1]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

import numpy as np

import pandas as pd


In [2]:
# carregar modelo word2vec
word2vec = KeyedVectors.load_word2vec_format('../skip_s50.txt')
vocab_size, embedding_size = word2vec.vectors.shape

In [3]:
# carregar dados de treinamento do CSV
df = pd.read_csv('../train.csv', header=None, names=['texto', 'classe'])
X = df['texto'].values
y = df['classe'].values

In [4]:
# tokenizar o texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# padronizar o tamanho das sequencias
max_length = max([len(text) for text in X])
X = pad_sequences(X, maxlen=max_length, padding='post')

In [5]:
# construir modelo
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights=[word2vec.vectors], input_length=max_length, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dense(7, activation='softmax'))

In [6]:
# compilar modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# separar dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 7)
y_val = to_categorical(y_val, 7)

In [8]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

In [9]:
# treinar modelo
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping_monitor])

Epoch 1/30
26/26 [==============================] - 4s 38ms/step - loss: 1.6112 - accuracy: 0.3879 - val_loss: 1.1048 - val_accuracy: 0.5665
Epoch 2/30
26/26 [==============================] - 0s 12ms/step - loss: 0.8218 - accuracy: 0.6416 - val_loss: 0.6446 - val_accuracy: 0.8030
Epoch 3/30
26/26 [==============================] - 0s 14ms/step - loss: 0.4460 - accuracy: 0.8005 - val_loss: 0.4116 - val_accuracy: 0.8473
Epoch 4/30
26/26 [==============================] - 1s 39ms/step - loss: 0.2975 - accuracy: 0.8461 - val_loss: 0.3490 - val_accuracy: 0.8818
Epoch 5/30
26/26 [==============================] - 1s 24ms/step - loss: 0.2584 - accuracy: 0.8929 - val_loss: 0.2815 - val_accuracy: 0.8916
Epoch 6/30
26/26 [==============================] - 0s 15ms/step - loss: 0.2094 - accuracy: 0.9249 - val_loss: 0.2644 - val_accuracy: 0.9310
Epoch 7/30
26/26 [==============================] - 0s 15ms/step - loss: 0.2014 - accuracy: 0.9310 - val_loss: 0.2955 - val_accuracy: 0.9261
Epoch 8/30
26